In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

# Model and API Key

In [2]:
# Load environment variables.
load_dotenv()

# Set the model name for our LLMs.
OPENAI_MODEL = "gpt-3.5-turbo"
# Store the API key in a variable.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# List Parser

In [3]:
# Additional imports for list parser.
from langchain.output_parsers import CommaSeparatedListOutputParser

In [4]:
# Initialize the model.
llm=ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0.9)

# Initialize the output parser.
parser = CommaSeparatedListOutputParser()

# Get the output format instructions.
instructions = parser.get_format_instructions()
print(instructions)

print()

# Define a query as a string, combining with the instructions.
query = "List 3 cat breeds." + "\n\n" + instructions

# Pass the query to the invoke method, and print the result.
result = llm.invoke(query)
print(result.content)

print()

# Parse the result, store it, and print it.
data = parser.parse(result.content)
print(data)

Your response should be a list of comma separated values, eg: `foo, bar, baz`

Persian, Siamese, Maine Coon

['Persian', 'Siamese', 'Maine Coon']


# Structured Parser

In [5]:
# Additional imports for structured parser.
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

In [6]:
# Initialize the model.
llm=ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0.9)

# Define the schemas for our parser.
schemas = [
    ResponseSchema(name="country", description="the country"),
    ResponseSchema(name="capital", description="the capital")
]

# Initialize the output parser using the schema.
parser = StructuredOutputParser.from_response_schemas(schemas)

# Get the output format instructions and print them.
instructions = parser.get_format_instructions()
print(instructions)

print()

# Define a query as a string, combining with the instructions.
query = "Name a country and its capital." + "\n\n" + instructions

# Pass the query to the invoke method, and print the result.
result = llm.invoke(query)
print(result.content)

print()

# Parse the result, store it, and print it.
data = parser.parse(result.content)
print(data)
print(data["country"])
print(data["capital"])

print()

# Define a new query using the parsed output.
query = f"What are three tourist attractions in {data['capital']}?"

# Pass the query to the invoke method, and print the result.
result = llm.invoke(query)
print(result.content)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"country": string  // the country
	"capital": string  // the capital
}
```

```json
{
	"country": "France",
	"capital": "Paris"
}
```

{'country': 'France', 'capital': 'Paris'}
France
Paris

1. Eiffel Tower: One of the most iconic landmarks in the world, the Eiffel Tower is a must-visit attraction in Paris. Visitors can take in breathtaking views of the city from its observation decks, enjoy a meal at one of its restaurants, or simply marvel at its stunning architecture.

2. Louvre Museum: As the world's largest art museum, the Louvre is a cultural landmark and a paradise for art lovers. Its vast collection includes famous masterpieces like the Mona Lisa and Venus de Milo, housed in a former royal palace. Visitors can spend hours exploring its various galleries and admiring the incredible artworks.

3. Notre-Dame Cathedral: A masterpiece of

# Class-based Parser

In [7]:
# Additional imports for Pydantic parser.
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import Field, BaseModel

In [8]:
# Initialize the model.
llm=ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0.9)

# Define the class for our parsed responses.
class Country(BaseModel):
    name: str = Field(description="the country name")
    capital: str = Field(description="the capital")
    population: int = Field(description="the country's population")

# Initialize the output parser using the schema.
parser = PydanticOutputParser(pydantic_object=Country)

# Get the output format instructions and print them.
instructions = parser.get_format_instructions()
print(instructions)

print()

# Define a query as a string, combining with the instructions.
query = "Name any country, its capital, and the country's population." + "\n\n" + instructions

# Pass the query to the invoke method, and print the result.
result = llm.invoke(query)
print(result.content)

print()

# Parse the result, store it, and print it.
data = parser.parse(result.content)
print(data)
print(data.name)
print(data.capital)
print(data.population)

print()

# Define a new query using the parsed output.
query = f"What are three tourist attractions in {data.capital}?"

# Pass the query to the invoke method, and print the result.
result = llm.invoke(query)
print(result.content)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"title": "Name", "description": "the country name", "type": "string"}, "capital": {"title": "Capital", "description": "the capital", "type": "string"}, "population": {"title": "Population", "description": "the country's population", "type": "integer"}}, "required": ["name", "capital", "population"]}
```

{
  "name": "United States",
  "capital": "Washington, D.C.",
  "population": 331002651
}

name='United States' capital='Washington, D.C.' population=331002651
United States
Washington, D.C.
331002651

Three tourist at